# Balance Log Explorer

Visualize the most recent balance attempts captured with `tools/balance_plot.py`. Compatible with both `motor_deg` (legacy) and `base_deg` (virtual encoder) logs.

In [ ]:
from __future__ import annotations

from pathlib import Path
from typing import Iterable, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

LOG_DIR = (Path('..') / 'logs').resolve()
LOG_DIR.mkdir(parents=True, exist_ok=True)
MAX_RECENT_ATTEMPTS = 5
print(f"Log directory: {LOG_DIR}")

In [ ]:
# Import helper utilities from tools/balance_plot.py
import sys
from importlib import import_module

repo_root = Path.cwd()
if (repo_root / 'balance_plot.py').exists():
    sys.path.append(str(repo_root))
if (repo_root / 'tools' / 'balance_plot.py').exists():
    sys.path.append(str(repo_root / 'tools'))

balance_logger = import_module('balance_plot')
find_recent_logs = getattr(balance_logger, 'find_recent_logs')
load_log_dataframe = getattr(balance_logger, 'load_log_dataframe')

RECENT_LOGS = find_recent_logs(limit=MAX_RECENT_ATTEMPTS, directory=LOG_DIR)
print(f"Loaded {len(RECENT_LOGS)} log(s) (limit={MAX_RECENT_ATTEMPTS})")
RECENT_LOGS

In [ ]:
def select_base_column(df: pd.DataFrame) -> tuple[str | None, str]:
    if 'base_deg' in df.columns:
        return 'base_deg', 'Base Angle (deg)'
    if 'motor_deg' in df.columns:
        return 'motor_deg', 'Motor Angle (deg)'
    return None, ''


def plot_attempt(path: Path) -> None:
    df = load_log_dataframe(path)
    if df.empty:
        print(f"[WARN] {path.name} is empty; skipping plot")
        return

    fig, axes = plt.subplots(2, 1, figsize=(11, 7), sharex=True)

    axes[0].plot(df['time_s'], df['setpoint_deg'], label='Setpoint', color='C1', linewidth=1.2)
    axes[0].plot(df['time_s'], df['pendulum_deg'], label='Pendulum', color='C0', linewidth=1.2)
    axes[0].set_ylabel('Angle (deg)')
    axes[0].set_title(f'Pendulum Response — {path.name}')
    axes[0].legend(loc='upper right')

    axes[1].plot(df['time_s'], df['stepper_steps'], label='Stepper Steps', color='C2')
    base_col, base_label = select_base_column(df)
    if base_col and not df[base_col].isna().all():
        axes[1].plot(df['time_s'], df[base_col], label=base_label, color='C3', linestyle='--')
    axes[1].plot(df['time_s'], df['control_output'], label='Control Output', color='C4', linestyle=':')
    axes[1].set_ylabel('Steps / deg')
    axes[1].set_xlabel('Time (s)')
    axes[1].set_title('Base Feedback & Control Effort')
    axes[1].legend(loc='upper right')

    fig.tight_layout()
    plt.show()


def compute_metrics(df: pd.DataFrame, threshold_deg: float = 5.0) -> dict[str, float]:
    angles = df['pendulum_deg'].to_numpy()
    times = df['time_s'].to_numpy()
    metrics = {
        'peak_abs_deg': float(np.nanmax(np.abs(angles))) if angles.size else float('nan'),
        'samples': int(df.shape[0]),
    }
    settling_time = float('nan')
    if angles.size:
        for idx, t in enumerate(times):
            if np.all(np.abs(angles[idx:]) <= threshold_deg):
                settling_time = float(t - times[0])
                break
    metrics['settling_time_s'] = settling_time
    return metrics

In [ ]:
if not RECENT_LOGS:
    print('[INFO] No logs yet. Run tools/balance_plot.py to capture sessions.')
else:
    for path in RECENT_LOGS:
        plot_attempt(path)

In [ ]:
summary_records: List[dict[str, float | str]] = []
if RECENT_LOGS:
    for path in RECENT_LOGS:
        df = load_log_dataframe(path)
        if df.empty:
            continue
        metrics = compute_metrics(df)
        metrics['file'] = path.name
        summary_records.append(metrics)

metrics_df = pd.DataFrame(summary_records)
metrics_df

In [ ]:
def extract_gain_summary(paths: Iterable[Path]) -> pd.DataFrame:
    rows = []
    for path in paths:
        df = load_log_dataframe(path)
        if df.empty:
            continue
        sample = df.iloc[0]
        rows.append({
            'file': path.name,
            'kp': float(sample['kp_balance']),
            'ki': float(sample['ki_balance']),
            'kd': float(sample['kd_balance']),
            'km': float(sample['kp_motor']),
            'origin': sample['origin'],
            'reason': sample['reason'],
            'session_id': int(sample['session_id']),
            'device_ms': int(sample['device_start_ms']),
            'timestamp': sample['host_start_iso']
        })
    return pd.DataFrame(rows)

GAIN_TABLE = extract_gain_summary(RECENT_LOGS)
GAIN_TABLE

In [ ]:
from IPython.display import Markdown, display

if not RECENT_LOGS:
    display(Markdown(f"**No log files found in** `{LOG_DIR}`."))
else:
    for path in RECENT_LOGS[::-1]:
        df = load_log_dataframe(path)
        if df.empty:
            continue
        gains = df.loc[0, ['kp_balance','ki_balance','kd_balance','kp_motor']].astype(float)
        origin = df.loc[0, 'origin']
        reason = df.loc[0, 'reason']
        session_id = int(df.loc[0, 'session_id'])
        device_ms = int(df.loc[0, 'device_start_ms'])
        timestamp = df.loc[0, 'host_start_iso']
        header = '\n'.join([
            f"### {path.name}",
            f"- Session ID: {session_id} (device ms: {device_ms})",
            f"- Timestamp: {timestamp}",
            f"- Origin: `{origin}` → Reason: `{reason}`",
            f"- Gains: Kp = {gains['kp_balance']:.4f}, Ki = {gains['ki_balance']:.4f}, Kd = {gains['kd_balance']:.4f}, Km = {gains['kp_motor']:.4f}",
        ])
        display(Markdown(header))
        plot_attempt(path)
